In [1]:
import glob
import numpy as np
import pandas as pd


In [2]:
col = ["time","s_stat","e_stat","type_car","tv","number"]
def loading(path):
    df = pd.read_csv(path)
    df.columns = col
    df1 = df[(df.type_car == "31")].reset_index(drop=True)
    return df1

def preprocessing(df):
    # processing time string
    con = pd.DataFrame(df.mins.str.split(" ",2).tolist(), columns = ['day','time'])
    con1 = pd.DataFrame(con.day.str.split("/",3).tolist(), columns = ['year','mon','day'])
    con2 = pd.DataFrame(con.time.str.split(":",2).tolist(), columns = ['hr','mins'])
    con2.hr = pd.to_numeric(con2.hr, errors = 'coerce')
    con2.mins = pd.to_numeric(con2.mins, errors = 'coerce')
    df = pd.concat([con1, df],axis=1)
    df["mins"] = (con2.hr*60 + con2.mins)/5
    
    df.year = pd.to_numeric(df.year, errors = 'coerce')
    df.mon = pd.to_numeric(df.mon, errors = 'coerce')
    df.day = pd.to_numeric(df.day, errors = 'coerce')
    df.number = pd.to_numeric(df.number, errors = 'coerce')
    df.clock = pd.to_numeric(df.clock, errors = 'coerce')
    df.speed = pd.to_numeric(df.speed, errors = 'coerce')
    return df


In [3]:
path = 'M04A'
allfile = glob.glob(path + "/*.csv")
df_t = pd.DataFrame()
list_ = []

for file_ in allfile:
    df = loading(file_)
    list_.append(df)
df_t = pd.concat(list_).reset_index(drop=True)
df_t.tail()

,time,s_stat,e_stat,type_car,tv,number
1246747,2017/09/09 23:55,05F0528N,05F0438N,31,327,6
1246748,2017/09/09 23:55,05F0309S,05F0439S,31,483,14
1246749,2017/09/09 23:55,05FR113S,05F0439S,31,109,4
1246750,2017/09/09 23:55,05F0439S,05F0494S,31,209,6
1246751,2017/09/09 23:55,05F0438N,05FR143N,31,104,5


In [4]:
path = 'M05A'
allfile = glob.glob(path + "/*.csv")
df_v = pd.DataFrame()
list_ = []

for file_ in allfile:
    df = loading(file_)
    list_.append(df)
df_v = pd.concat(list_).reset_index(drop=True)
df_v.tail()

,time,s_stat,e_stat,type_car,tv,number
1246747,2017/09/09 23:55,05F0528N,05F0438N,31,99,6
1246748,2017/09/09 23:55,05F0309S,05F0439S,31,97,14
1246749,2017/09/09 23:55,05FR113S,05F0439S,31,105,4
1246750,2017/09/09 23:55,05F0439S,05F0494S,31,95,6
1246751,2017/09/09 23:55,05F0438N,05FR143N,31,107,5


In [5]:
df = pd.merge(df_t, df_v, on=['time','s_stat','e_stat','type_car','number'], how='inner')
df.columns = ['mins','s_stat','e_stat','type_car','clock','number','speed']
df = preprocessing(df)
df = df.drop(["type_car"], axis=1)
df.to_csv("M0405.csv", index=False)
df.head()

,year,mon,day,mins,s_stat,e_stat,clock,number,speed
0,2017,9,1,0.0,01F0017N,01F0005N,45,26,95
1,2017,9,1,0.0,01F0029N,01F0017N,45,26,95
2,2017,9,1,0.0,01F0005S,01F0017S,43,21,99
3,2017,9,1,0.0,01F0061N,01F0029N,117,33,98
4,2017,9,1,0.0,01F0017S,01F0029S,45,21,95


In [6]:
df["time_"] = df.mon*100000 + df.day*1000 + df.mins
df = df.drop(["year","mon","day","mins"], axis=1)
df.head()

,s_stat,e_stat,clock,number,speed,time_
0,01F0017N,01F0005N,45,26,95,901000.0
1,01F0029N,01F0017N,45,26,95,901000.0
2,01F0005S,01F0017S,43,21,99,901000.0
3,01F0061N,01F0029N,117,33,98,901000.0
4,01F0017S,01F0029S,45,21,95,901000.0


In [7]:
loc = pd.read_csv("location_clean.csv")

con = pd.DataFrame(loc.num1.str.split("-",2).tolist(), columns = ['highway','a'])
loc = pd.concat([loc, con.highway], axis=1)
loc = loc.drop(["num1"], axis=1)

print (loc.dir.value_counts())
print (con.highway.value_counts())
loc.tail()

N    166
S    165
Name: dir, dtype: int64
03F    152
01F    146
01H     15
05F     14
03A      4
Name: highway, dtype: int64


,dir,lat,lon,num2,highway
326,N,24.842639,121.789286,05F0287N,05F
327,N,24.823706,121.786214,05F0309N,05F
328,N,24.733917,121.782350,05FR143N,05F
329,N,24.710922,121.789597,05F0438N,05F
330,N,24.632717,121.807167,05F0528N,05F


> F:一般高速公路, A:台北聯絡線(台北－深坑)　, H:汐止-楊梅高架道路

In [8]:
loc.columns = ['dir', 'lat', 'lon', 's_stat', 'highway']
df_ = pd.merge(df, loc, how='left', on=['s_stat'])
loc.columns = ['dir', 'lat', 'lon', 'e_stat', 'highway']
df_ = pd.merge(df_, loc, how='left', on=['e_stat'])

df_.tail()

,s_stat,e_stat,clock,number,speed,time_,dir_x,lat_x,lon_x,highway_x,dir_y,lat_y,lon_y,highway_y
1246747,05F0528N,05F0438N,327,6,99,909287.0,N,24.632717,121.807167,05F,N,24.710922,121.789597,05F
1246748,05F0309S,05F0439S,483,14,97,909287.0,S,24.823706,121.786214,05F,S,24.711028,121.789478,05F
1246749,05FR113S,05F0439S,109,4,105,909287.0,S,24.733944,121.781989,05F,S,24.711028,121.789478,05F
1246750,05F0439S,05F0494S,209,6,95,909287.0,S,24.711028,121.789478,05F,S,24.662581,121.799850,05F
1246751,05F0438N,05FR143N,104,5,107,909287.0,N,24.710922,121.789597,05F,N,24.733917,121.782350,05F


#### 交流道不同

In [9]:
change = df_[(df_.highway_x != df_.highway_y)].reset_index(drop=True)

f = {'time_':['count']}
g = change.groupby(["s_stat","e_stat"]).aggregate(f).reset_index()
g.columns = ["s_stat","e_stat","num"]

print ("size of change highway :", len(set(g.s_stat)), len(set(g.e_stat)))
change.tail()

size of change highway : 78 78


,s_stat,e_stat,clock,number,speed,time_,dir_x,lat_x,lon_x,highway_x,dir_y,lat_y,lon_y,highway_y
383611,01F3640N,03F3854S,722,2,109,909287.0,N,22.658892,120.332081,01F,S,22.769197,120.441839,03F
383612,01F3686S,03F4142N,0,0,0,909287.0,S,22.618206,120.336853,01F,N,22.579586,120.526161,03F
383613,01F3686S,03F4168S,880,3,100,909287.0,S,22.618206,120.336853,01F,S,22.557186,120.524558,03F
383614,03F0158S,05F0000S,41,24,43,909287.0,S,25.036533,121.621386,03F,S,25.035089,121.622931,05F
383615,03F0201N,05F0000S,175,12,78,909287.0,N,25.007375,121.596381,03F,S,25.035089,121.622931,05F


#### 交流道相同、南北向不同

In [10]:
change = df_[(df_.dir_x != df_.dir_y)&(df_.highway_x == df_.highway_y)].reset_index(drop=True)

f = {'time_':['count']}
g = change.groupby(["s_stat","e_stat"]).aggregate(f).reset_index()
g.columns = ["s_stat","e_stat","num"]

print ("size of change direction :", g.shape[0])
change.tail()

size of change direction : 12


,s_stat,e_stat,clock,number,speed,time_,dir_x,lat_x,lon_x,highway_x,dir_y,lat_y,lon_y,highway_y
31099,03F2336N,03F2336S,0,0,0,909287.0,N,23.882031,120.709186,03F,S,23.882031,120.709186,03F
31100,03F2747S,03F2747N,2783,1,5,909287.0,S,23.618203,120.558567,03F,N,23.618203,120.558567,03F
31101,03F2777N,03F2777S,0,0,0,909287.0,N,23.606792,120.532511,03F,S,23.606792,120.532511,03F
31102,03F3187S,03F3187N,2406,2,3,909287.0,S,23.296692,120.404164,03F,N,23.296692,120.404164,03F
31103,03F3211N,03F3211S,923,1,9,909287.0,N,23.279281,120.390253,03F,S,23.279281,120.390253,03F
